In [1]:
import os
import pathlib

p_project = str(pathlib.Path(os.getcwd()).parents[1])

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 300)

In [3]:
adm=pd.read_csv(p_project + "/temp/pic/processed/admissions_processed.csv")

We now consider the prescriptions dataset. We select only the patients present in the cleaned admission file

In [4]:
presc=pd.read_csv(p_project + "/data/pic/PRESCRIPTIONS.csv.gz", compression='gzip')

#Restrict the dataset to the previously selected admission ids only.
adm_ids=list(adm["HADM_ID"])
presc=presc.loc[presc["HADM_ID"].isin(adm_ids)]

print("Number of patients remaining in the database: ")
print(presc["SUBJECT_ID"].nunique())

Number of patients remaining in the database: 
3207


In [ ]:
# wait


In [6]:
#Select entries whose drug name is in the list from the paper.
drugs_list=["Aspirin","Bisacodyl","Docusate Sodium","D5W","Humulin-R Insulin","Potassium Chloride","Magnesium Sulfate","Metoprolol Tartrate","Sodium Chloride 0.9%  Flush","Pantoprazole"]
presc2=presc.loc[presc["DRUG"].isin(drugs_list)]

print("Number of patients remaining in the database: ")
print(presc2["SUBJECT_ID"].nunique())

Number of patients remaining in the database: 
22569


### Units Cleaning

#### 1) In amounts

In [8]:
#Verification that all input labels have the same amounts units.

#Units correction
presc2=presc2.drop(presc2.loc[presc2["DOSE_UNIT_RX"].isnull()].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["DRUG"]=="Acetaminophen")&(presc2["DOSE_UNIT_RX"]!="mg")].index).copy()
presc2.loc[(presc2["DRUG"]=="D5W")&(presc2["DOSE_UNIT_RX"]=="ml"),"DOSE_UNIT_RX"]="mL"
presc2=presc2.drop(presc2.loc[(presc2["DRUG"]=="D5W")&(presc2["DOSE_UNIT_RX"]!="mL")].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["DRUG"]=="Heparin")&(presc2["DOSE_UNIT_RX"]!="UNIT")].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["DRUG"]=="Insulin")&(presc2["DOSE_UNIT_RX"]!="UNIT")].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["DRUG"]=="Magnesium Sulfate")&(presc2["DOSE_UNIT_RX"]!="gm")].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["DRUG"]=="Potassium Chloride")&(presc2["DOSE_UNIT_RX"]!="mEq")].index).copy()
presc2.loc[(presc2["DRUG"]=="Sodium Chloride 0.9%  Flush")&(presc2["DOSE_UNIT_RX"]=="ml"),"DOSE_UNIT_RX"]="mL"
presc2=presc2.drop(presc2.loc[(presc2["DRUG"]=="Bisacodyl")&(presc2["DOSE_UNIT_RX"]!="mg")].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["DRUG"]=="Humulin-R Insulin")&(presc2["DOSE_UNIT_RX"]!="UNIT")].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["DRUG"]=="Pantoprazole")&(presc2["DOSE_UNIT_RX"]!="mg")].index).copy()

print(presc2.groupby("DRUG")["DOSE_UNIT_RX"].value_counts())

DRUG                         DOSE_UNIT_RX
Aspirin                      mg              14980
Bisacodyl                    mg              22577
D5W                          mL              66611
Docusate Sodium              mg              26319
Humulin-R Insulin            UNIT             1841
Magnesium Sulfate            gm              45892
Metoprolol Tartrate          mg              34200
Pantoprazole                 mg              15670
Potassium Chloride           mEq             97807
Sodium Chloride 0.9%  Flush  mL              43488
Name: DOSE_UNIT_RX, dtype: int64


### Check for outliers

#### 1) In amounts

In [9]:
type(presc2["DOSE_VAL_RX"])

#We need to transform the value columns in float type.
original_num_entries=len(presc2.index)
#First transform the ranges (xx-yy) as the mean of the ranges.
range_df=presc2.loc[presc2["DOSE_VAL_RX"].str.contains("-")].copy()
range_df["First_digit"]=range_df["DOSE_VAL_RX"].str.split("-").str[0].astype(float)
range_df["Second_digit"]=range_df["DOSE_VAL_RX"].str.split("-").str[1]
range_df.loc[range_df["Second_digit"]=="",'Second_digit']=range_df.loc[range_df["Second_digit"]=="",'First_digit']
range_df["Second_digit"]=range_df["Second_digit"].astype(float)
range_df.head()
range_df["mean"]=(range_df["First_digit"]+range_df["Second_digit"])/2
range_df["DOSE_VAL_RX"]=range_df["mean"]
range_df.drop(columns=["First_digit","Second_digit","mean"],inplace=True)

#Now remove the entries with the - from the original df and force conversion to float.
presc3=presc2.drop(presc2.loc[presc2["DOSE_VAL_RX"].str.contains("-")].index).copy()
presc3["DOSE_VAL_RX"]=pd.to_numeric(presc2["DOSE_VAL_RX"], errors="coerce")
presc3.dropna(subset=["DOSE_VAL_RX"],inplace=True)

presc2=presc3.append(range_df)

print("Lost entries in the process : {}".format(original_num_entries-len(presc2.index)))

Lost entries in the process : 3


In [10]:
presc2.groupby("DRUG")["DOSE_VAL_RX"].describe()

,count,mean,std,min,25%,50%,75%,max
DRUG,,,,,,,,
Aspirin,14980.0,193.658081,121.437073,0.0,81.0,81.0,325.0,975.0
Bisacodyl,22576.0,10.103871,14.648323,5.0,10.0,10.0,10.0,2210.0
D5W,66609.0,272.359829,217.836532,0.0,250.0,250.0,250.0,3250.0
Docusate Sodium,26319.0,100.993579,10.463674,100.0,100.0,100.0,100.0,400.0
Humulin-R Insulin,1841.0,100.451657,14.342874,1.0,100.0,100.0,100.0,500.0
Magnesium Sulfate,45892.0,2.504406,0.842189,0.0,2.0,2.0,3.0,20.0
Metoprolol Tartrate,34200.0,22.998220,22.735655,0.0,5.0,12.5,25.0,200.0
Pantoprazole,15670.0,41.466752,7.638066,0.0,40.0,40.0,40.0,120.0
Potassium Chloride,97807.0,36.655157,15.723778,0.0,20.0,40.0,40.0,420.0


In [11]:
#Remove all entries whose rate is more than 4 std away from the mean.
presc_desc=presc2.groupby("DRUG")["DOSE_VAL_RX"].describe()
name_list=list(presc_desc.loc[presc_desc["count"]!=0].index)
for label in name_list:
    presc2=presc2.drop(presc2.loc[(presc2["DRUG"]==label)&(presc2["DOSE_VAL_RX"]>(presc_desc.loc[label,"mean"]+4*presc_desc.loc[label,"std"]))].index).copy()

print("Number of patients remaining in the database: ")
print(presc2["SUBJECT_ID"].nunique())
print("Number of datapoints remaining in the database: ")
print(len(presc2.index))

Number of patients remaining in the database: 
22569
Number of datapoints remaining in the database: 
368289


In [12]:
presc2.groupby("DRUG")["DOSE_VAL_RX"].describe()

,count,mean,std,min,25%,50%,75%,max
DRUG,,,,,,,,
Aspirin,14979.0,193.605918,121.273184,0.0,81.0,81.0,325.0,650.0
Bisacodyl,22575.0,10.006423,0.433214,5.0,10.0,10.0,10.0,40.0
D5W,66567.0,270.661377,207.030172,0.0,250.0,250.0,250.0,1000.0
Docusate Sodium,26069.0,100.000000,0.000000,100.0,100.0,100.0,100.0,100.0
Humulin-R Insulin,1830.0,99.580055,6.123123,1.0,100.0,100.0,100.0,100.0
Magnesium Sulfate,45876.0,2.502446,0.831430,0.0,2.0,2.0,3.0,5.0
Metoprolol Tartrate,34036.0,22.403900,21.051369,0.0,5.0,12.5,25.0,100.0
Pantoprazole,15087.0,39.975078,0.823581,0.0,40.0,40.0,40.0,40.0
Potassium Chloride,97784.0,36.629213,15.556814,0.0,20.0,40.0,40.0,90.0


In [13]:
presc2['CHARTTIME']=pd.to_datetime(presc2["STARTDATE"], format='%Y-%m-%d %H:%M:%S')

#To avoid confounding labels with labels from other tables, we add "drug" to the name
presc2["DRUG"]=presc2["DRUG"]+" Drug"

In [14]:
presc2.to_csv(p_project + "/temp/pic/processed/prescriptions_processed.csv")